# Basic RAG over an existing vector database

In [ ]:
#!pip install sentence_transformers
#!pip install ipywidgets

Select **model** for embeddings. 

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
#!pip install chromadb

Access existing database

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="some_facts",
    embedding_function=embeddings,
    persist_directory="./chroma_some_facts",  # Where to save data locally, remove if not necessary
)

Create retriever

In [ ]:
retriever = vector_store.as_retriever()

Make a search in the vectorstor (only for testing purposes)

In [ ]:
question = "Can I steal a bank?"
docs = vector_store.similarity_search(question)
len(docs)

In [ ]:
docs[0]

RAG Chain

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [ ]:
# Question
chain.invoke("Can I steal a bank?")